# Algorytm Hopcrofta-Karpa

Algorytm ten wykorzystuje ścieżki powiększające do znalezienia największego skojarzenia w grafie dwudzielnym.

*Ścieżka powiększająca* to ścieżka która zaczyna się i kończy na wierzchołkach wcześniej nieskojarzonych i na przemian występują krawędzie spoza skojarzenia i ze skojarzenia.

Algorytm wykorzystuje BFS i DFS do znalezienia najkrótszych ścieżek rozszerzających.

### Jak działa algorytm?
Zakładamy tu, że graf dwudzielny składa się z dwóch zbiorów wierzchołków: U i V.

1. Na początek inicjujemy zmienną matching, która będzie mówiła o tym ile jest par w obecnym skojarzeniu.
2. Następnie wykonujemy BFS, aby znaleźć najkrótszą ścieżkę rozszerzającą:
    - tworzymy kolejkę (póki co pustą)
    - dla każdego wierzchołka u w U, jeśli jest on nieskojarzony, ustawiamy jego odległość od początku ścieżki augmentującej na 0 i dodajemy go do kolejki. W przeciwnym razie ustawiamy jego odległość na $\infty$.
    - ustawiamy minimalną odległość od dowolnego wolnego wierzchołka w U do końca augmentującej ścieżki na $\infty$.
    - przetwarzamy wierzchołki z kolejki: dla każdego wierzchołka u w kolejce (jeśli jest on bliżej początku ścieżki powiększającej niż koniec - jeśli nie jest to jeszcze koniec ścieżki powiększającej) przeglądamy jego sąsiadów v w V. Jeśli odległość sparowanego wierzchołka z v jest $\infty$ (nie został on jeszcze przetworzony), aktualizujemy ją (self.dist[self.pairV[v]] = self.dist[u] + 1) i dodajemy ten wierzchołek do kolejki.
    - BFS zwraca True, jeśli znaleziono przynajmniej jedną rozszerzającą ścieżkę (czyli self.dist[None] nie jest nieskończonością).
3. Po wykonaniu BFS, przechodzimy do przeglądania wszystkich wierzchołków u z U. Jeśli u jest wolny, wywołujemy DFS(u).
    - dla każdego sąsiada wierzchołka u - v sprawdzamy, czy sparowany z nim wierzchołek jest następny w kolejności przetwarzania (self.dist[self.pairV[v]] == self.dist[u] + 1).
    - jeśli tak, rekurencyjnie wywołujemy DFS na tym wierzchołku
    - jeśli DFS(self.pairV[v]) zwróci True, oznacza to, że znaleźliśmy ścieżkę powiększającą. Aktualizujemy parowanie, ustawiając self.pairV[v] na u i self.pairU[u] na v, i zwracamy True.
    - jeśli nie znaleźliśmy ścieżki rozszerzającej z u, ustawiamy self.dist[u] na $\infty$, aby ten wierzchołek mógł być uwzględniony w następnych ścieżkach powiększających i zwracamy False.
4. Jeśli DFS znajdzie augmentującą ścieżkę (zwróci True), zwiększamy matching o 1.
5. Na końcu funkcji hopcroft_karp zwracamy wartość matching, która reprezentuje maksymalne skojarzenie. Zmienne self.pairU i self.pairV przechowują skojarzenie (parowanie wierzchołków z U i V).

## Wyjściowy graf dwudzielny
<img src='1.png' width='350'/>

## Zaczynamy od pustego grafu
<img src='2.png' width='350'/>

## Algorytm BFS
Szukamy najprostszego połączenia (łączymy wolny wierzchołek z U z pierwszym wolnym sąsiadem z V)
<img src='3.png' width='350'/>

Dla wolnych wierzchołków z U szukamy sąsiadów z V
<img src='5.png' width='350'/>

Dla sąsiadów z V szukamy skojarzonych z nimi wierzchołków z U...
<img src='6.png' width='350'/>

... dla nich szukamy sąsiadów z V (i tak aż dojdziemy do wolnych wierzchołków z V)
<img src='7.png' width='350'/>

## Algorytm DFS
Zaczynając od niesparowanego wierzchołka z V szukamy ścieżki powiększającej, która prowadzi do niesparowanego wierzzchołka z U
<img src='8.png' width='350'/>

Usuwamy tą ścieżkę z drzewa i powtarzamy dla kolejnego wolnego wierzchołka z V
<img src='9.png' width='350'/>

Kontynuujemy do ostatniego wolnego wierzchołka z V
<img src='10.png' width='350'/>

Aktualizujemy dotychczasowe skojarzenie o nowo znalezione ścieżki rozszerzające
<img src='11.png' width='350'/>
<img src='12.png' width='350'/>

Na takim grafie wykonujemy algorytm BFS. Jeśli dalej są ścieżki powiększające - powtarzamy procedurę. Jeśli nie - zwracamy skojarzenie.

In [1]:
import numpy as np
from random import random
from copy import deepcopy
from queue import PriorityQueue, deque

In [2]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph
        self.weighted = False

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_array(cls, graph: np.array, nodes: list = None):
        if nodes is None:
            nodes = [*range(1, len(graph) + 1)]
        return cls.from_dict(
            cls._array_to_dict(graph, nodes)
        )

    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Generates a graphs from a text file, where each line defines one edge.\n
        Filename is a file path.
        """
        graph = Graph()
        file = open(filename, "r")
        for line in file:
            words = line.strip().split()
            if len(words) == 1:
                graph.add_node(words[0])
            elif len(words) == 2: # two words -- unweighed graph
                if directed:
                    graph.add_arc([words[0], words[1]])
                else:
                    graph.add_edge([words[0], words[1]])
            elif len(words) > 2: # more than two words - labeled graph
                graph.weighted = True
                if directed:
                    graph.add_arc([words[0], words[1]])
                    graph.graph[words[0]][-1] = (words[1], words[2]) # process label
                else:
                    graph.add_edge([words[0], words[1]])
                    graph.graph[words[0]][-1] = (words[1], words[2])
                    graph.graph[words[1]][-1] = (words[0], words[2])
        file.close()
        return graph

    @staticmethod
    def random_graph(nodes_num: int, prob: float):
        """
        Generates a random graph provided a number of nodes and probability of generating an edge.
        """
        rand_graph = Graph()
        for i in range(1, nodes_num + 1):
            rand_graph.add_node(i)
            for j in range(1, i):
                if random() < prob:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(nodes_num: int):
        """
        Generates a cycle provided a number of nodes.
        """
        cycle = Graph()
        nodes = [*range(1, nodes_num + 1)]
        for i in nodes:
            cycle.add_edge([nodes[i-2], nodes[i-1]])
        cycle.graph = dict(sorted(cycle.graph.items()))
        return cycle

    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")
        file.write(str(self))
        file.close()

    def nodes(self) -> list:
        """
        Returns list of nodes of a graph.
        """
        return [*self.graph.keys()]

    def array(self) -> np.array:
        """
        Returns the graph in array form.
        """
        return self._dict_to_array(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    def add_node(self, node):
        """
        Adds a node to a graph.
        """
        if node not in self.graph:
            self.graph[node] = []

    def del_node(self, node):
        """
        Recursively removes a node from a graph.
        """
        if node in self.graph:
            self.graph.pop(node)
            for key in [*self.graph.keys()]:
                if node in self.graph[key]:
                    self.graph[key].remove(node)

    def add_arc(self, arc: list):
        """
        Adds arc to a graph provided a list of nodes.
        """
        u, v = arc
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Adds edge to a graph provided a list of nodes.
        """
        u, v = edge
        if u == v:
            raise ValueError("Pętla!")
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    
    def bfs(self):                                                   # algorytm BFS
        queue = deque()                                              # tworzymy kolejkę
        for u in self.U:                                             # dla każdego wierzchołka z U
            if self.pairU[u] is None:                                # jeżeli u jest nie jest sparowany
                self.dist[u] = 0                                     # ustawiamy jego odległość u od początku ścieżki rozszerzającej na 0
                queue.append(u)                                      # i dodajemy go do kolejki
            else:                                                    # jeśli u jest już sparowany
                self.dist[u] = float('inf')                          # to ustawiamy jego odległość na inf (nie będzie on początkiem ścieżki augmentującej)
            
        self.dist[None] = float('inf')                               # minimalna odległość od dowolnego wolnego wierzchołka w U do końca augmentującej ścieżki
        
        while queue:                                                 # dopóki kolejka jest niepusta
            u = queue.popleft()                                      # bierzemy pierwszy element z kolejki - u
            if self.dist[u] < self.dist[None]:                       # jeśli u jest bliżej początku ścieżki augmentującej niż koniec
                for v in self.graph[u]:                              # to dla każdego v, który sąsiaduje z u
                    if self.dist[self.pairV[v]] == float('inf'):     # jeśli sparowany wierzchołek z v nie został odwiedzony
                        self.dist[self.pairV[v]] = self.dist[u] + 1  # to zmieniamy jego odległość na odległość u +1
                        queue.append(self.pairV[v])                  # i dodajemy do kolejki 
        return self.dist[None] != float('inf')                       # jeśli istnieje ścieżka rozszerzająca - funkcja zwraca TRUE
    
    def dfs(self, u):                                                # algorytm DFS 
        if u is not None:                                            # jeśli u istnieje
            for v in self.graph[u]:                                  # dla v, który sąsiaduje z u
                if self.dist[self.pairV[v]] == self.dist[u] + 1:     # jeśli sparowanegy wierzchołek z v jest następny w kolejce przetwarzania
                    if self.dfs(self.pairV[v]):                      # to wywołujemy na nim DFS, jeśli znajdzie ścieżkę powiększającą dla niego
                        self.pairV[v] = u                            # aktualizujemy skojarzenie
                        self.pairU[u] = v
                        return True                                  # true oznacza, że została znaleziona ścieżka rozszerzająca
            self.dist[u] = float('inf')                              # jeśli u jest sparowany - ustawiamy jego odległość na inf
            return False                                             # zwracamy False = nie znaleziono ścieżki rozszerzającej
        return True                                                  # zwracamy True = znaleniono ścieżkę rozszerzającą
    
    def hopcroft_karp(self):                                         # algorytm Hopcrofta - Karpa
        matching = 0                                                 # ustawiamy matching na 0
        while self.bfs():                                            # jeśli BFS zwróci True
            for u in self.U:                                         # dla każdego wierzchołka z U
                if self.pairU[u] is None:                            # jeśli jest on nieskojarzony
                    if self.dfs(u):                                  # jeśli DFS(u) zwróci True (znajdzie ścieżkę rozszerzającą)
                        matching += 1                                # zwiększamy matching o 1
        return matching                                              # zwracamy matching - największe skojarzenie

In [3]:
def random_bipartite_graph(n,p):
    ver1=[]                                                 # zaczątek listy wierzchołków U
    ver2=[]                                                 # zaczątek listy wierzchołków V
    graph=Graph.from_dict({})                               # zaczątek grafu
    
    for i in range(1,n+1):                                 
        ver1.append(i)                                      # tworzymy n wierzcholków z U
        ver2.append(i+n)                                    # tworzymy n wierzcholków z V
    
    for vertex1 in ver1:                                    # dla każdego wierzchołka z U
        for vertex2 in ver2:                                # dla każdego wierzchołka z V
            if random()<p:                                  # jeśli wylosowana wartość jest mniejsza niż zadane p
                graph.add_edge([vertex1,vertex2])           # dodajemy krawędź między u i v
    
    for v in ver1: 
        if v not in graph.graph:
            graph.add_node(v)                               # dodajemy wierzchołki izolowane z U
    for v in ver2:
        if v not in graph.graph:
            graph.add_node(v)                               # dodajemy wierzchołki izolowane z V
    graph.U=ver1                                            # wierzchołki U   
    graph.V=ver2                                            # wierzchołki U 
    graph.pairU = {u: None for u in graph.U}                # wierzchołki skojarzone z U (domyślnie wszystkie ustawiamy na None)
    graph.pairV = {v: None for v in graph.V}                # wierzchołki skojarzone z V (domyślnie wszystkie ustawiamy na None)
    graph.dist={}                                           # zaczątek słownika odległości
    return graph                                            # zwracamy graf

In [127]:
g=random_bipartite_graph(5, 1/2)
g.graph

{1: [6, 7, 10],
 6: [1, 2, 3, 4],
 7: [1, 4, 5],
 10: [1, 5],
 2: [6],
 3: [6, 9],
 9: [3, 5],
 4: [6, 7, 8],
 8: [4, 5],
 5: [7, 8, 9, 10]}

In [128]:
g.hopcroft_karp()

5

In [129]:
g.pairU

{1: 10, 2: 6, 3: 9, 4: 7, 5: 8}

In [130]:
g.pairV

{6: 2, 7: 4, 8: 5, 9: 3, 10: 1}